In [1]:
#  mongodb://localhost:27017/

In [4]:
import pyodbc



In [17]:
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=localhost\\SQLEXPRESS;"
    "DATABASE=Retail Business;"
    "Trusted_Connection=yes;"
)


In [18]:
def get_db_connection():
    """
    This function creates and returns a NEW connection
    to the SQL Server database.

    Why we use a function:
    - Avoid repeating connection code everywhere
    - Easier to maintain and update
    - Cleaner and more professional code
    """

    # 1. Use pyodbc to open a connection to SQL Server
    # The 'conn_str' contains:
    # - Driver
    # - Server name
    # - Database name
    # - Authentication method
    connection = pyodbc.connect(conn_str)

    # 2. Return the open connection
    # The calling code will:
    # - Create a cursor
    # - Execute SQL commands
    # - Close the connection when finished
    return connection

In [32]:
# ====================================================
# Function to Get All Customers from Database
# ====================================================

def get_customers():
    """
    This function reads ALL customers from the database and returns them as a list of dictionaries.

    Steps it performs:
    1. Connects to the SQL Server database
    2. Executes a SELECT query to get all customer records
    3. Converts each database row into a Python dictionary (object)
    4. Returns a list of customer dictionaries
    """
    conn = None
    try:
        # 1. Open a connection to the database
        conn = get_db_connection()

        # 2. Create a cursor
        cursor = conn.cursor()

        # 3. Write and execute the SQL SELECT query
        cursor.execute("""
            SELECT CustomerID, Name, Gender, BirthDate, Phone, Email, City
            FROM dbo.Customers
        """)

        # 4. Fetch all rows returned by the query
        rows = cursor.fetchall()

        # 5. Create an empty list to store customer dictionaries
        customers = []

        # 6. Loop through each row and convert it into a Python dictionary
        for row in rows:
            customer = {
                "customer_id": row.CustomerID,
                "name": row.Name,
                "gender": row.Gender,
                # 👇 تحويل BirthDate لنص (string) عشان MongoDB يقدر يخزنه
                "BirthDate": row.BirthDate.strftime("%Y-%m-%d") if row.BirthDate else None,
                "phone": row.Phone,
                "email": row.Email,
                "city": row.City
            }
            customers.append(customer)

        # 7. Return True to indicate success, and the list of customer dictionaries
        return True, customers

    except Exception as e:
        # 8. If something goes wrong, return False and the error message
        return False, str(e)

    finally:
        # 9. ALWAYS close the database connection
        if conn is not None:
            conn.close()


In [33]:
# ====================================================
# Example: Get All Customers from Database
# ====================================================

# 1. Call the function to get all customers
# - get_customers() returns two things:
#   1. success → True if the function worked, False if there was an error
#   2. customers → list of customer dictionaries if success=True, or error message if success=False
success, customers = get_customers()

# 2. Check if the function was successful
if success:
    # ✅ Function worked, we have a list of customers
    print("Customers found:", len(customers))  # Show total number of customers

    # Loop through each customer and print its details
    for c in customers:
        # c is a dictionary representing one customer
        print(c)

else:
    # ❌ Something went wrong (e.g., database connection error)
    # - customers variable contains the error message
    print("Error:", customers)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [34]:
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId

In [35]:
# define connection string
client = MongoClient('mongodb://localhost:27017/')

In [36]:
# Retail Business
db = client["Retail_Business"]



In [37]:
customers_col = db["Customers"]


In [38]:
def insert_many_customers(customers_list):
    """
    customers_list: list of dictionaries, each containing customer info, e.g.,
    [
        {"Name": "Alice Smith", "Age": 25, "Gender": "Female", "Region": "Cairo"},
        {"Name": "Bob Johnson", "Age": 30, "Gender": "Male", "Region": "Giza"}
    ]
    """
    try:
        # Attempt to insert multiple customer documents
        result = customers_col.insert_many(customers_list)

        # Check if documents were actually inserted
        if result.inserted_ids:
            return True, f"{len(result.inserted_ids)} customers inserted successfully with IDs: {result.inserted_ids}"
        else:
            return False, "No customers were inserted"

    except Exception as e:
        # Handle any error during insert operation
        return False, str(e)

In [39]:

# Call the function and receive the result
success, message = insert_many_customers(customers)

# Check if the insert operation was successful
if success:
    print(message)
else:
    print("Error inserting customers:", message)

20000 customers inserted successfully with IDs: [ObjectId('69738139f4a5170580a5d540'), ObjectId('69738139f4a5170580a5d541'), ObjectId('69738139f4a5170580a5d542'), ObjectId('69738139f4a5170580a5d543'), ObjectId('69738139f4a5170580a5d544'), ObjectId('69738139f4a5170580a5d545'), ObjectId('69738139f4a5170580a5d546'), ObjectId('69738139f4a5170580a5d547'), ObjectId('69738139f4a5170580a5d548'), ObjectId('69738139f4a5170580a5d549'), ObjectId('69738139f4a5170580a5d54a'), ObjectId('69738139f4a5170580a5d54b'), ObjectId('69738139f4a5170580a5d54c'), ObjectId('69738139f4a5170580a5d54d'), ObjectId('69738139f4a5170580a5d54e'), ObjectId('69738139f4a5170580a5d54f'), ObjectId('69738139f4a5170580a5d550'), ObjectId('69738139f4a5170580a5d551'), ObjectId('69738139f4a5170580a5d552'), ObjectId('69738139f4a5170580a5d553'), ObjectId('69738139f4a5170580a5d554'), ObjectId('69738139f4a5170580a5d555'), ObjectId('69738139f4a5170580a5d556'), ObjectId('69738139f4a5170580a5d557'), ObjectId('69738139f4a5170580a5d558'), O

In [41]:
# ====================================================
# Function to Get All Branches from Database
# ====================================================

def get_Branches():
    """
    This function reads ALL Branches from the database and returns them as a list of dictionaries.

    Steps it performs:
    1. Connects to the SQL Server database
    2. Executes a SELECT query to get all customer records
    3. Converts each database row into a Python dictionary (object)
    4. Returns a list of customer dictionaries
    """
    conn = None
    try:
        # 1. Open a connection to the database
        conn = get_db_connection()

        # 2. Create a cursor
        cursor = conn.cursor()

        # 3. Write and execute the SQL SELECT query
        cursor.execute("""
            SELECT BranchID, BranchName, City, Address
            FROM dbo.Branches
        """)

        # 4. Fetch all rows returned by the query
        rows = cursor.fetchall()

        # 5. Create an empty list to store customer dictionaries
        branches = []

        # 6. Loop through each row and convert it into a Python dictionary
        for row in rows:
            branches = {
                "branch_id": row.BranchID,
                "branch_name": row.BranchName,
                "city": row.City,
                "address": row.Address

                }
            branches.append(branches)

        # 7. Return True to indicate success, and the list of customer dictionaries
        return True, branches

    except Exception as e:
        # 8. If something goes wrong, return False and the error message
        return False, str(e)

    finally:
        # 9. ALWAYS close the database connection
        if conn is not None:
            conn.close()


In [42]:
# ====================================================
# Example: Get All Branches from Database
# ====================================================

# 1. Call the function to get all Branches
# - get_Branches() returns two things:
#   1. success → True if the function worked, False if there was an error
#   2. Branches → list of Branches dictionaries if success=True, or error message if success=False
success, branches = get_Branches()

# 2. Check if the function was successful
if success:
    # ✅ Function worked, we have a list of Branches
    print("Branches found:", len(branches))  # Show total number of Branches

    # Loop through each Branches and print its details
    for c in branches:
        # c is a dictionary representing one customer
        print(c)

else:
    # ❌ Something went wrong (e.g., database connection error)
    # - Branches variable contains the error message
    print("Error:", branches)

Error: 'dict' object has no attribute 'append'


In [43]:
"""
select custname, orderdate, totalamount, productname, qty
from customers
join orders on
join
"""

'\nselect custname, orderdate, totalamount, productname, qty\nfrom customers\njoin orders on\njoin\n'

In [51]:
def get_all_orders():
    conn = None
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        cursor.execute("""
            SELECT
                c.CustomerID,
                c.Name AS customer_name,
                c.Gender,
                c.BirthDate,
                c.City AS customer_city,
                b.BranchID,
                b.BranchName,
                b.City AS branch_city,
                b.Address AS branch_address,
                s.SaleID,
                s.SaleDate,
                s.TotalAmount,
                p.ProductID,
                p.ProductName,
                p.Category,
                p.Price,
                sd.Quantity
            FROM Sales s
            JOIN Customers c ON s.CustomerID = c.CustomerID
            JOIN Branches b ON s.BranchID = b.BranchID
            JOIN SaleDetails sd ON s.SaleID = sd.SaleID
            JOIN Products p ON sd.ProductID = p.ProductID
        """)

        rows = cursor.fetchall()
        orders = []

        for row in rows:
            order = {
                "customer": {
                    "id": row.CustomerID,
                    "name": row.customer_name,
                    "gender": row.Gender,
                    "birthdate": row.BirthDate.strftime("%Y-%m-%d") if row.BirthDate else None,
                    "city": row.customer_city
                },
                "branch": {
                    "id": row.BranchID,
                    "name": row.BranchName,
                    "city": row.branch_city,
                    "address": row.branch_address
                },
                "sale": {
                    "id": row.SaleID,
                    "date": row.SaleDate.strftime("%Y-%m-%d") if row.SaleDate else None,
                    "total_amount": float(row.TotalAmount)  # 👈 هنا حول Decimal لـ float
                },
                   "product": {
                    "id": row.ProductID,
                    "name": row.ProductName,
                    "category": row.Category,
                    "price": float(row.Price),
                    "quantity": row.Quantity
                }
            }
            orders.append(order)

        return True, orders

    except Exception as e:
        return False, str(e)
    finally:
        if conn is not None:
            conn.close()


In [52]:
import pymongo
from pymongo import MongoClient

In [53]:
client = MongoClient('mongodb://localhost:27017/')


In [54]:
db = client["Retail_Business"]

In [55]:
orders_col = db["Orders"]

In [56]:
# ====================================================
# Function to insert many orders into MongoDB
# ====================================================

def insert_many_orders(orders_list):
    """
    orders_list: list of dictionaries, each containing full order info from SQL
    """
    try:
        # Attempt to insert multiple order documents
        result = orders_col.insert_many(orders_list)

        # Check if documents were actually inserted
        if result.inserted_ids:
            return True, f"{len(result.inserted_ids)} orders inserted successfully!"
        else:
            return False, "No orders were inserted"

    except Exception as e:
        # Handle any error during insert operation
        return False, str(e)


In [57]:
# 1. جلب كل الـ orders من SQL
success, orders = get_all_orders()

# 2. إدخالها في MongoDB
if success:
    success_insert, message = insert_many_orders(orders)
    if success_insert:
        print(message)
    else:
        print("Error inserting orders:", message)
else:
    print("Error fetching orders:", orders)


200000 orders inserted successfully!
